In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"balasahebtambe","key":"4fff30fe9a0515682892f2615073aef0"}'}

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d deadskull7/fer2013

Dataset URL: https://www.kaggle.com/datasets/deadskull7/fer2013
License(s): CC0-1.0
 94% 91.0M/96.6M [00:00<00:00, 118MB/s]
100% 96.6M/96.6M [00:00<00:00, 109MB/s]


In [9]:
!unzip fer2013.zip

Archive:  fer2013.zip
  inflating: fer2013.csv             


In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the CSV file
data = pd.read_csv("fer2013.csv")

# View the structure of the data
print(data.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [11]:
data.shape

(35887, 3)

In [12]:
# Preprocess the data
def preprocess_data(data):
    # Split the pixels column and convert it into a NumPy array
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []

    for pixel_sequence in pixels:
        face = np.array([int(pixel) for pixel in pixel_sequence.split(' ')], dtype='float32')
        face = face.reshape(width, height, 1)
        faces.append(face)

    faces = np.array(faces)

    # Normalize the pixel values (0-255 to 0-1)
    faces = faces / 255.0

    # Convert labels to categorical format (one-hot encoding)
    emotions = to_categorical(data['emotion'], num_classes=7)

    return faces, emotions

In [13]:
# Preprocess the training data
faces, emotions = preprocess_data(data)

In [14]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.2, random_state=42)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
def create_model():
    model = Sequential()

    # Convolutional layer 1
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional layer 2
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional layer 3
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flattening the output
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))  # Dropout to prevent overfitting

    # Output layer (7 classes for each emotion)
    model.add(Dense(7, activation='softmax'))

    return model

In [16]:
# Instantiate the model
model = create_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       1,048,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,420,295 (5.42 MB)

 Trainable params: 1,420,295 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2)

Epoch 1/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.2429 - loss: 1.8213 - val_accuracy: 0.3776 - val_loss: 1.6154
Epoch 2/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.3804 - loss: 1.5836 - val_accuracy: 0.4702 - val_loss: 1.4022
Epoch 3/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.4620 - loss: 1.4012 - val_accuracy: 0.4995 - val_loss: 1.3334
Epoch 4/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.5070 - loss: 1.2905 - val_accuracy: 0.5275 - val_loss: 1.2508
Epoch 5/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5353 - loss: 1.2159 - val_accuracy: 0.5455 - val_loss: 1.2154
Epoch 6/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5610 - loss: 1.1578 - val_accuracy: 0.5554 - val_loss: 1.1849
Epoch 7/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.5832 - loss: 1.0899 - val_accuracy: 0.5596 - val_loss: 1.1804
Epoch 8/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6151 - loss: 1.0283 - val_ac

In [19]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test accuracy: {test_accuracy * 100:.2f}%")

225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5723 - loss: 2.2706
Test accuracy: 57.02%


In [20]:
# Save the trained model
model.save('facial_expression_model.h5')

In [21]:
model.save('facial_expression_model.keras')